In [1]:
!pip install python-constraint


  Preparing metadata (setup.py) ... done
  Created wheel for python-constraint: filename=python_constraint-1.4.0-py2.py3-none-any.whl size=24057 sha256=eadde7597f3696e239993f4aabc5de11d7804737c5bdf120f93d49d7c9dd1d9a
  Stored in directory: /root/.cache/pip/wheels/2e/f2/2b/cb08b5fe129e4f69b7033061f256e5c551b0aa1160c2872aee
Successfully built python-constraint


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


break


In [47]:
def read_faculty(file_path):
    with open(file_path, 'r') as file:
        faculty_data = [line.strip().split(';') for line in file.readlines()]
    return [(faculty, preferences.split(','), int(hours)) for faculty, preferences, hours in faculty_data if preferences]

# ... (Rest of the code)

def read_faculty(file_path):
    with open(file_path, 'r') as file:
        faculty_data = [line.strip().split(';') for line in file.readlines()]
    return [(faculty, preferences.split(','), int(hours)) for faculty, preferences, hours in faculty_data]

def read_courses(file_path):
    with open(file_path, 'r') as file:
        courses_data = [line.strip().split(';') for line in file.readlines()]

    course_slots = []
    for data in courses_data:
        if len(data) == 2:
            course, slots = data
            course = course.strip()
            slots_parts = slots.strip().split()
            if len(slots_parts) > 0:
                slots_value = int(slots_parts[0])
                course_slots.append((course, slots_value))

    return course_slots

def initialize_timetable(sections_data, days, slots):
    timetable = {}

    for section, courses in sections_data:
        # Calculate the target number of slots for each subject in the section
        target_slots_per_subject = 9
        subjects_count = len(courses)
        total_target_slots = subjects_count * target_slots_per_subject

        # Initialize the timetable for the section
        timetable[section] = {day: {slot: None for slot in slots} for day in days}

        # Distribute the target slots evenly among subjects
        slots_per_subject = total_target_slots // subjects_count

        for course in courses:
            for day in days:
                for slot in slots[:slots_per_subject]:
                    # Assign the course to the slot
                    timetable[section][day][slot] = (None, course)

    return timetable


def assign_courses_to_faculty(faculty_data, courses_data, timetable):
    faculty_preferences = {faculty: preferences for faculty, preferences, _ in faculty_data}
    faculty_assignments = {faculty: [] for faculty in faculty_preferences}

    for course, _ in courses_data:
        assigned = False
        for faculty, preferences in faculty_preferences.items():
            if course in preferences and len(faculty_assignments[faculty]) < 2:
                faculty_assignments[faculty].append(course)
                assigned = True
                break

        if not assigned:
            # Assign to any faculty with fewer than 2 assignments
            for faculty in faculty_preferences:
                if len(faculty_assignments[faculty]) < 2:
                    faculty_assignments[faculty].append(course)
                    break

    return faculty_assignments


import random

import random

# ... (previous code)
def schedule_courses(timetable, faculty_assignments, sections_data, days, slots):
    for section, courses in sections_data:
        for course in courses:
            # Check if the course is already assigned to both sections
            sec_01_assigned = any(course in timetable['SEC-01'][day].values() for day in days)
            sec_02_assigned = any(course in timetable['SEC-02'][day].values() for day in days)

            if not sec_01_assigned or not sec_02_assigned:
                # Assign the course to both sections if not already assigned
                for faculty in faculty_assignments:
                    if len(faculty_assignments[faculty]) < 2:
                        # Find an available slot for the course in SEC-01
                        if not sec_01_assigned:
                            target_section = 'SEC-01'
                        elif not sec_02_assigned:
                            target_section = 'SEC-02'
                        else:
                            break  # Both sections already assigned

                        faculty_assignments[faculty].append(course)

                        # Find an available slot for the course in the target section's timetable
                        for day in days:
                            for slot in slots:
                                if timetable[target_section][day][slot] is None:
                                    timetable[target_section][day][slot] = (faculty, course)
                                    break

                        break


def print_subject_section_faculty(sections_data, faculty_assignments):
    print(" Time Table Generated ")
    print("Subject\tSection\tFaculty")
    for section, courses in sections_data:
        for course in courses:
            assigned_faculty = None
            for faculty, assigned_courses in faculty_assignments.items():
                if course in assigned_courses:
                    assigned_faculty = faculty
                    break
            print(f"{course}\t{section}\t{assigned_faculty}")


print_subject_section_faculty(sections_data, faculty_assignments)



def adjust_slots_for_sections(timetable, sections_data, days, slots):

    target_slots = 9

    for section, schedule in timetable.items():
        assigned_slots = sum(1 for day_slots in schedule.values() for slot, assigned_course in day_slots.items() if assigned_course is not None)
        while assigned_slots > target_slots:
            # Find a day and slot with assigned course in the current section
            occupied_slots = [(day, slot) for day, day_slots in schedule.items() for slot, assigned_course in day_slots.items() if assigned_course is not None]
            if occupied_slots:
                day, slot = random.choice(occupied_slots)

                # Find an empty slot in any section
                while True:
                    target_section = random.choice(list(timetable.keys()))
                    target_day, target_slot = random.choice(list(schedule[target_section].keys())), random.choice(slots)

                    if timetable[target_section][target_day][target_slot] is None:
                        # Move the course to the empty slot
                        timetable[target_section][target_day][target_slot] = timetable[section][day][slot]
                        timetable[section][day][slot] = None
                        assigned_slots -= 1
                        break
            else:
                break



def calculate_total_hours(timetable, faculty_name):
    total_hours = 0
    for section, schedule in timetable.items():
        for _, slots in schedule.items():
            for _, assignment in slots.items():
                if assignment and assignment[0] == faculty_name:
                    total_hours += 1
    return total_hours

def apply_faculty_constraints(timetable, faculty_assignments, faculty_data, days, slots):
    penalties = 0

    for faculty, courses, hours in faculty_data:
        consecutive_penalty = 10
        excess_course_penalty = 10
        excess_hours_penalty_1 = 5
        excess_hours_penalty_2 = 12
        excess_hours_penalty_invalid = float('inf')
        wrong_subject_penalty = 10

        consecutive_back_to_back_penalty = 10

        consecutive_back_to_back = False

        for i in range(len(courses)):
            course = courses[i]
            section, day, slot = find_course_slot(timetable, course)

            # Handle the case where find_course_slot returns None
            if section is None or day is None or slot is None:
                continue

            # Penalty for back-to-back allotment
            if i > 0:
                prev_course = courses[i - 1]
                _, prev_day, prev_slot = find_course_slot(timetable, prev_course)
                if day == prev_day and slots.index(slot) == slots.index(prev_slot) + 1:
                    penalties += consecutive_penalty

                    # Check for consecutive back-to-back courses for the same faculty
                    if timetable[section][day][prev_slot][0] == faculty:
                        consecutive_back_to_back = True

            # Penalty for excess course allocation
            if i >= 2:
                penalties += excess_course_penalty

            # Update the timetable with assigned course
            timetable[section][day][slot] = (faculty, course)

        # Penalty for excess hours
        total_hours = calculate_total_hours(timetable, faculty)
        if total_hours > hours:
            if total_hours - hours == 1:
                penalties += excess_hours_penalty_1
            elif total_hours - hours == 2:
                penalties += excess_hours_penalty_2
            else:
                penalties = excess_hours_penalty_invalid

        # Penalty for consecutive back-to-back courses for the same faculty
        if consecutive_back_to_back:
            penalties += consecutive_back_to_back_penalty

        # Penalty for wrong subject adoption
        for course in courses:
            section, _, _ = find_course_slot(timetable, course)
            if course not in faculty_assignments[faculty] and section is not None and section != '':
                penalties += wrong_subject_penalty

    return penalties

def find_course_slot(timetable, course):
    for section, schedule in timetable.items():
        for day, slots in schedule.items():
            for slot, assignment in slots.items():
                if assignment and assignment[1] == course:
                    return section, day, slot
    # If the course is not found in the timetable, return None
    return None, None, None

def output_timetable_csv(timetable, days, slots):
    output_path_section = "/content/drive/MyDrive/TimeTable/SectionT.csv"
    output_path_faculty = "/content/drive/MyDrive/TimeTable/FacultyT.csv"

    with open(output_path_section, 'w') as file_section, open(output_path_faculty, 'w') as file_faculty:
        # Write the header for both section and faculty timetables
        file_section.write("Faculty")
        for day in days:
            for slot in slots:
                file_section.write(f"\t{day}{slot}")
        file_section.write("\n")

        # Write the timetable data for sections
        for section, schedule in timetable.items():
            file_section.write(f"{section}")
            for day in days:
                for slot in slots:
                    assignment = schedule[day][slot]
                    if assignment:
                        faculty, course = assignment
                        file_section.write(f"\t{section} ({course})")
                    else:
                        file_section.write("\t")
            file_section.write("\n")

        # Write the header for the faculty timetable
        file_faculty.write("Faculty")
        for day in days:
            for slot in slots:
                file_faculty.write(f"\t{day}{slot}")
        file_faculty.write("\n")

        # Write the timetable data for faculties
        for faculty in set([faculty for section_schedule in timetable.values() for _, slots in section_schedule.items() for _, assignment in slots.items() if assignment is not None for faculty, _ in [assignment]]):
            file_faculty.write(f"{faculty}")
            for day in days:
                for slot in slots:
                    found = False
                    for section, schedule in timetable.items():
                        assignment = schedule[day][slot]
                        if assignment and assignment[0] == faculty:
                            file_faculty.write(f"\t{section} ({assignment[1]})")
                            found = True
                            break
                    if not found:
                        file_faculty.write("\t")
            file_faculty.write("\n")


# Define your file paths
sections_file_path = "/content/drive/MyDrive/TimeTable/DATA/Sections.txt"
faculty_file_path = "/content/drive/MyDrive/TimeTable/DATA/Faculty.txt"
courses_file_path = "/content/drive/MyDrive/TimeTable/DATA/Courses.txt"

# Read data from files
sections_data = read_sections(sections_file_path)
faculty_data = read_faculty(faculty_file_path)
courses_data = read_courses(courses_file_path)

# Define your days and slots
days = ["Mon", "Tue", "Wed", "Thu", "Fri"]
slots = ["1", "2", "3", "4", "5", "6", "7", "8"]

# Initialize timetable
timetable = initialize_timetable(sections_data, days, slots)

# Assign courses to faculty
# Assign courses to faculty
faculty_assignments = assign_courses_to_faculty(faculty_data, courses_data, timetable)


# Schedule courses in the timetable
schedule_courses(timetable, faculty_assignments, sections_data, days, slots)

# Apply faculty constraints and calculate penalties
penalties = apply_faculty_constraints(timetable, faculty_assignments, faculty_data, days, slots)

# Output timetables to CSV files
output_timetable_csv(timetable, days, slots)


 Time Table Generated 
Subject	Section	Faculty
ToC	SEC-01	GS	
DBMS	SEC-01	SS	
AI	SEC-01	GS	
NLP	SEC-01	PB	
ML	SEC-01	PB	
ToC	SEC-02	GS	
DBMS	SEC-02	SS	
AI	SEC-02	GS	
NLP	SEC-02	PB	
ML	SEC-02	PB	
